In [ ]:
import time
from picamera2 import Picamera2, Preview
from picamera2.controls import Controls
import cv2
import numpy as np
import matplotlib.pyplot as plt
import RPi.GPIO as GPIO
from time import sleep
import tifffile

In [ ]:
led = 23
GPIO.setmode(GPIO.BCM)
GPIO.setup(led, GPIO.OUT)

picam2 = Picamera2()

# Set camera controls
controls = {"ExposureTime": 1500000, #microseconds
            "AnalogueGain":1.0, # 1 = ISO 100
            "AeEnable": False, # Auto exposure and Gain
            "AwbEnable": False,# Auto white Balance
            "FrameDurationLimits": (114,239000000)} #Min/Max frame duration
# Setup config parameters
preview_config = picam2.create_preview_configuration(raw={"size": picam2.sensor_resolution, "format": "SBGGR12",},
                                                     controls = controls) 
print(preview_config)

In [ ]:

picam2.start_preview(Preview.QTGL)
picam2.configure(preview_config)

GPIO.output(led, GPIO.HIGH) # Turn on LED

picam2.start() # Start Camera
 
time.sleep(2)

#Capture image in unpacked RAW format 12bit dynamic range (16bit array)
raw = picam2.capture_array("raw").view(np.uint16)

GPIO.output(led, GPIO.LOW) # Turn off LED

#print(picam2.stream_configuration("raw"))
print(picam2.capture_metadata())

picam2.stop_preview()
picam2.stop()

#plt.imshow(raw, cmap="gray")

In [ ]:
center_crop = raw[1000:2000, 2000:3000]

#Get color channels in bayer order (BGGR)
red = center_crop[1::2,1::2]
green1 = center_crop[0::2,1::2]
green2 = center_crop[1::2,0::2]
green = np.add(green1,green2)/2

#Make histogram for red and green channel # Set camera controls to have good pixel saturation
Colors=("red","green")
Channel_ids=(red,green)
for channel_id, c in zip(Channel_ids,Colors):
    histogram, bin_edges=np.histogram(channel_id,bins=4095, range=(0,4095))
    plt.plot(bin_edges[0:-1],histogram,color=c)
plt.title("Red_Green histogram")
plt.xlabel("Pixel intensity")
plt.ylabel("Pixel Frequency")
plt.show()
print("Histogram Succes")

print(np.mean(red))
print(np.mean(green))
